# WebBaseLoader

## 概覽

```WebBaseLoader``` 是 LangChain 中專門用來處理網頁內容的文件載入器。

它使用 ```BeautifulSoup4``` 函式庫來有效解析網頁，並透過 ```SoupStrainer``` 及其他 ```bs4``` 參數提供彈性的解析選項。

本教學將示範如何使用 ```WebBaseLoader``` 來：
1. 有效載入與解析網頁文件
2. 使用 ```BeautifulSoup``` 的選項自訂解析行為
3. 靈活處理不同結構的網頁內容

### 目錄 

- [概覽](#overview)
- [環境設置](#environment-setup)
- [載入網頁文件](#load-web-based-documents)
- [使用 alazy_load 並行載入多個網址](#load-multiple-urls-concurrently-with-alazy_load)
- [載入 XML 文件](#load-xml-documents)
- [使用代理伺服器載入網頁文件](#load-web-based-document-using-proxies)
- [使用 MarkItDown 輕量載入網頁內容](#simple-web-content-loading-with-markitdown)

### 參考資料

- [WebBaseLoader API 文件](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.web_base.WebBaseLoader.html)
- [BeautifulSoup4 官方文件](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
----

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- ```langchain-opentutorial``` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [```langchain-opentutorial```](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [9]:
%%capture --no-stderr
%pip install langchain-opentutorial markitdown

In [10]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langchain_community",
    ],
    verbose=False,
    upgrade=False,
)

In [11]:
from dotenv import load_dotenv
load_dotenv()

True

## 載入網頁文件

```WebBaseLoader``` 是專為載入網頁文件而設計的載入器。

它使用 ```bs4```（BeautifulSoup）函式庫來解析網頁內容。

主要特點包括：
- 使用 ```bs4.SoupStrainer``` 來指定要解析的 HTML 元素。
- 可透過 ```bs_kwargs``` 參數傳入額外的 ```bs4.SoupStrainer``` 相關參數。

更多細節請參考其 API 文件。

## 匯入 BeautifulSoup 的解析模組
#### import bs4

### 匯入 LangChain 的 WebBaseLoader，用來從網頁上讀取文件
#### from langchain_community.document_loaders import WebBaseLoader

### ----------------------------------------
### 使用 WebBaseLoader 載入網頁內容（新聞文章）
### ----------------------------------------
loader = WebBaseLoader(
    # 指定目標網頁網址：這是一篇來自 TechCrunch 的新聞文章
    web_paths=(
        "https://techcrunch.com/2024/12/28/google-ceo-says-ai-model-gemini-will-the-companys-biggest-focus-in-2025/",
    ),

    # 使用 bs4（BeautifulSoup）的參數指定只解析特定 HTML 標籤區塊
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",  # 只處理 <div> 標籤
            attrs={
                # 只抓取具有以下 class 屬性的內容區塊（即文章主體）
                "class": [
                    "entry-content wp-block-post-content is-layout-constrained wp-block-post-content-is-layout-constrained"
                ]
            },
        )
    ),

    # 自訂 request headers 的 User-Agent，模擬瀏覽器行為（避免被網站阻擋）
    header_template={
        "User_Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36",
    },
)

# ----------------------------------------
### 執行 loader，實際發送請求並解析頁面內容為文件物件（Document）
# ----------------------------------------
docs = loader.load()

# ----------------------------------------
### 輸出載入的文件數量（應為 1 筆）
# ----------------------------------------
print(f"Number of documents: {len(docs)}")

### 顯示第一份 Document 的內容（包含 page_content 與 metadata）
docs[0]


⸻

## ✅ 這段程式的重點摘要：

### 部分	說明
WebBaseLoader	用於載入特定網頁的內容並轉為 Document 格式
bs_kwargs	限制 BeautifulSoup 只解析文章主體的 HTML 區塊
header_template	模擬正常使用者瀏覽器發出的 User-Agent，避免被網站防爬蟲阻擋
docs = loader.load()	實際發送 HTTP 請求並取得內容
docs[0]	取得解析後的第一份 Document（含文字與中繼資料）


In [12]:
# 匯入 BeautifulSoup 的解析模組
import bs4

# 匯入 LangChain 的 WebBaseLoader，用來從網頁上讀取文件
from langchain_community.document_loaders import WebBaseLoader

# ----------------------------------------
# 使用 WebBaseLoader 載入網頁內容（新聞文章）
# ----------------------------------------
loader = WebBaseLoader(
    # 指定目標網頁網址：這是一篇來自 TechCrunch 的新聞文章
    web_paths=(
        "https://techcrunch.com/2024/12/28/google-ceo-says-ai-model-gemini-will-the-companys-biggest-focus-in-2025/",
    ),

    # 使用 bs4（BeautifulSoup）的參數指定只解析特定 HTML 標籤區塊
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",  # 只處理 <div> 標籤
            attrs={
                # 只抓取具有以下 class 屬性的內容區塊（即文章主體）
                "class": [
                    "entry-content wp-block-post-content is-layout-constrained wp-block-post-content-is-layout-constrained"
                ]
            },
        )
    ),

    # 自訂 request headers 的 User-Agent，模擬瀏覽器行為（避免被網站阻擋）
    header_template={
        "User_Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36",
    },
)

# ----------------------------------------
# 執行 loader，實際發送請求並解析頁面內容為文件物件（Document）
# ----------------------------------------
docs = loader.load()

# ----------------------------------------
# 輸出載入的文件數量（應為 1 筆）
# ----------------------------------------
print(f"Number of documents: {len(docs)}")

# 顯示第一份 Document 的內容（包含 page_content 與 metadata）
docs[0]

Number of documents: 1


Document(metadata={'source': 'https://techcrunch.com/2024/12/28/google-ceo-says-ai-model-gemini-will-the-companys-biggest-focus-in-2025/'}, page_content='\nGoogle CEO Sundar Pichai reportedly told Google employees that 2025 will be a “critical” year for the company.\nCNBC reports that it obtained audio from a December 18 strategy meeting where Pichai and other executives put on ugly holiday sweaters and laid out their priorities for the coming year.\n\n\n\n\n\n\n\n\n“I think 2025 will be critical,” Pichai said. “I think it’s really important we internalize the urgency of this moment, and need to move faster as a company. The stakes are high.”\nThe moment, of course, is one where tech companies like Google are making heavy investments in AI, and often with mixed results. Pichai acknowledged that the company has some catching up to do on the AI side — he described the Gemini app (based on the company’s AI model of the same name) as having “strong momentum,” while also acknowledging “we h

To bypass SSL authentication errors, you can set the ```“verify”``` option.

In [13]:
# Bypass SSL certificate verification
loader.requests_kwargs = {"verify": False}

# Load documents from the web
docs = loader.load()
docs[0]

/opt/homebrew/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'techcrunch.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Document(metadata={'source': 'https://techcrunch.com/2024/12/28/google-ceo-says-ai-model-gemini-will-the-companys-biggest-focus-in-2025/'}, page_content='\nGoogle CEO Sundar Pichai reportedly told Google employees that 2025 will be a “critical” year for the company.\nCNBC reports that it obtained audio from a December 18 strategy meeting where Pichai and other executives put on ugly holiday sweaters and laid out their priorities for the coming year.\n\n\n\n\n\n\n\n\n“I think 2025 will be critical,” Pichai said. “I think it’s really important we internalize the urgency of this moment, and need to move faster as a company. The stakes are high.”\nThe moment, of course, is one where tech companies like Google are making heavy investments in AI, and often with mixed results. Pichai acknowledged that the company has some catching up to do on the AI side — he described the Gemini app (based on the company’s AI model of the same name) as having “strong momentum,” while also acknowledging “we h

You can also load multiple webpages at once. To do this, you can pass a list of **urls** to the loader, which will return a list of documents in the order of the **urls** passed.

In [14]:
# 假設您已經匯入了必要的 bs4 模組，如果沒有，您需要先 import bs4
import bs4 

# 初始化 WebBaseLoader，並設定其參數
loader = WebBaseLoader(
    web_paths=[
        # 這是一個包含目標網頁 URL 的列表。
        # WebBaseLoader 會嘗試從列表中的每個 URL 下載和解析內容。
        "https://techcrunch.com/2024/12/28/revisiting-the-biggest-moments-in-the-space-industry-in-2024/",
        "https://techcrunch.com/2024/12/29/ai-data-centers-could-be-distorting-the-us-power-grid/",
    ],
    bs_kwargs=dict(
        # bs_kwargs 允許您傳遞額外的關鍵字參數給 BeautifulSoup 解析器。
        # BeautifulSoup 是一個流行的 Python 函式庫，用於從 HTML 和 XML 文件中提取資料。

        # 'parse_only' 是 BeautifulSoup 的一個重要參數，它與 bs4.SoupStrainer 物件一起使用，
        # 用於指示解析器只處理 HTML 文件中符合特定條件的部分。
        # 這樣做的好處是：
        # 1. 效率提升：只解析相關部分可以顯著加快處理速度，特別是對於大型複雜的網頁。
        # 2. 精確提取：可以更精確地定位到您感興趣的內容，避免提取不必要的資訊（如導覽列、廣告、頁腳等）。
        parse_only=bs4.SoupStrainer(
            # bs4.SoupStrainer 的第一個參數是要篩選的 HTML 標籤名稱。
            # 在這裡，我們指定為 "div"，表示我們只對 <div> 標籤感興趣。
            "div",
            # attrs 參數是一個字典，用於指定標籤必須擁有的屬性及其值。
            # 在這個例子中，我們篩選 class 屬性。
            # class 屬性的值是一個列表，包含多個 CSS 類別名稱：
            # ["entry-content", "wp-block-post-content", "is-layout-constrained", "wp-block-post-content-is-layout-constrained"]
            # 這意味著 BeautifulSoup 只會解析那些 **同時擁有所有這些 CSS 類別** 的 <div> 標籤及其內部所有內容。
            # 這種設定通常用於定位網頁中的主要內容區域，例如文章主體。
            # 開發者通過檢查網頁原始碼，找出包含主要內容的特定 <div> 標籤所使用的 class 組合。
            attrs={"class": ["entry-content", "wp-block-post-content", "is-layout-constrained", "wp-block-post-content-is-layout-constrained"]},
        )
    ),
    header_template={
        # header_template 允許您自訂發送 HTTP 請求時的標頭。
        # 'User-Agent' 是一個常見的請求標頭，它告知伺服器發起請求的客戶端類型（例如瀏覽器類型和版本）。
        # 某些網站可能會根據 User-Agent 檢查請求的合法性，或者返回針對特定瀏覽器優化的內容。
        # 設定一個常見的瀏覽器 User-Agent 有助於模擬真實用戶訪問，避免被網站阻擋。
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36",
    },
)

# 使用配置好的 loader 從指定的網頁載入資料
# loader.load() 方法會執行以下操作：
# 1. 遍歷 web_paths 中的每個 URL。
# 2. 對每個 URL，使用 header_template 中定義的標頭發送 HTTP GET 請求。
# 3. 獲取網頁的 HTML 內容。
# 4. 使用 BeautifulSoup（以及 bs_kwargs 中指定的配置，特別是 parse_only）解析 HTML。
# 5. 將解析出的內容轉換為 LangChain 的 Document 物件。
# docs 將會是一個包含多個 Document 物件的列表，每個 Document 對應一個成功載入並解析的網頁（或其特定部分）。
docs = loader.load()

# 檢查並印出成功載入的文件數量
# len(docs) 會返回列表 docs 中的元素數量，即成功載入的 Document 數量。
print(len(docs))

2


Output the results fetched from the web.

In [15]:
print(docs[0].page_content[:500])
print("===" * 10)
print(docs[1].page_content[:500])

## 使用 ```alazy_load()``` 並行載入多個網址

你可以透過非同步載入來加快擷取與解析多個網址的速度。這種方式允許你同時提取多份文件，在遵守請求速率限制的同時提升效率。

### 重點說明：
- **請求速率限制**：```requests_per_second``` 參數用於控制每秒可發出的請求數量。本範例中設為 1，以避免對伺服器造成過大負擔。
- **非同步載入**：使用 ```alazy_load()``` 函式可非同步地載入文件，提升多網址處理效能。
- **Jupyter Notebook 相容性**：若在 Jupyter Notebook 中執行，需使用 ```nest_asyncio``` 套件來正確處理非同步任務。

以下程式碼範例將示範如何設定並非同步載入文件：

In [16]:
# only for jupyter notebook (asyncio)
import nest_asyncio

nest_asyncio.apply()

In [17]:
# Set the requests per second rate limit
loader.requests_per_second = 1

# Load documents asynchronously
# The aload() is deprecated and alazy_load() is used since the langchain 3.14 update)
docs=[]
async for doc in loader.alazy_load():
    docs.append(doc)

In [18]:
# Display loaded documents
docs

[Document(metadata={'source': 'https://techcrunch.com/2024/12/28/revisiting-the-biggest-moments-in-the-space-industry-in-2024/'}, page_content=''),
 Document(metadata={'source': 'https://techcrunch.com/2024/12/29/ai-data-centers-could-be-distorting-the-us-power-grid/'}, page_content='')]

## 載入 XML 文件

```WebBaseLoader``` 可透過指定不同的 ```BeautifulSoup``` 解析器來處理 XML 檔案。這在處理結構化 XML 內容（例如網站地圖或政府資料）時特別實用。

### 基本 XML 載入範例

以下範例示範如何從政府網站載入一份 XML 文件：

In [19]:
from langchain_community.document_loaders import WebBaseLoader

# Initialize loader with XML document URL
loader = WebBaseLoader(
    "https://www.govinfo.gov/content/pkg/CFR-2018-title10-vol3/xml/CFR-2018-title10-vol3-sec431-86.xml"
)

# Set parser to XML mode
loader.default_parser = "xml"

# Load and process the document
docs = loader.load()

### 節省記憶體的載入方式

為了處理大型文件，```WebBaseLoader``` 提供了兩種節省記憶體的載入方法：

1. ```lazy_load()``` - 一次載入一個頁面，適合逐頁處理  
2. ```alazy_load()``` - 非同步載入頁面，可提升多頁面處理效能

In [20]:
# Lazy Loading Example
pages = []
for doc in loader.lazy_load():
    pages.append(doc)

# Print first 100 characters and metadata of the first page
print(pages[0].page_content[:100])
print(pages[0].metadata)



10
Energy
3
2018-01-01
2018-01-01
false
Uniform test method for the measurement of energy efficien
{'source': 'https://www.govinfo.gov/content/pkg/CFR-2018-title10-vol3/xml/CFR-2018-title10-vol3-sec431-86.xml'}


In [21]:
# Async Loading Example
pages = []
async for doc in loader.alazy_load():
    pages.append(doc)

# Print first 100 characters and metadata of the first page
print(pages[0].page_content[:100])
print(pages[0].metadata)



10
Energy
3
2018-01-01
2018-01-01
false
Uniform test method for the measurement of energy efficien
{'source': 'https://www.govinfo.gov/content/pkg/CFR-2018-title10-vol3/xml/CFR-2018-title10-vol3-sec431-86.xml'}


## 使用代理伺服器載入網頁文件

有時候你可能需要透過代理伺服器來繞過 IP 限制或封鎖。

要使用代理，只需將代理設定字典傳遞給載入器（底層會由 ```requests``` 函式庫處理）。

### ⚠️ 注意事項：
- 請將 ```{username}```、```{password}``` 與 ```proxy.service.com``` 替換為你實際的代理帳號與伺服器資訊。
- 若未正確設定代理，可能會出現 **ProxyError** 或 **AuthenticationError** 等錯誤。

In [ ]:
loader = WebBaseLoader(
   "https://www.google.com/search?q=parrots",
   proxies={
       "http": "http://{username}:{password}:@proxy.service.com:6666/",
       "https": "https://{username}:{password}:@proxy.service.com:6666/",
   },
   # Initialize the web loader with proxy settings
   # Configure proxy for both HTTP and HTTPS requests
)

# Load documents using the proxy
docs = loader.load()

## 使用 ```MarkItDown``` 輕量載入網頁內容

與使用 ```BeautifulSoup4``` 進行進階 HTML 解析的 ```WebBaseLoader``` 不同，```MarkItDown``` 提供一種較為簡單直接的網頁內容載入方式。它會透過 HTTP 請求直接擷取網頁內容，並轉換為 Markdown 格式，過程中不進行細緻的 HTML 結構解析。

以下是使用 ```MarkItDown``` 載入網頁內容的基本範例：

In [23]:
from markitdown import MarkItDown

md = MarkItDown()
result = md.convert("https://techcrunch.com/2024/12/28/revisiting-the-biggest-moments-in-the-space-industry-in-2024/")
result_text = result.text_content

In [24]:
print(result_text[:1000])

[![](https://techcrunch.com/wp-content/uploads/2024/09/tc-lockup.svg) TechCrunch Desktop Logo](https://techcrunch.com)

[![](https://techcrunch.com/wp-content/uploads/2024/09/tc-logo-mobile.svg) TechCrunch Mobile Logo](https://techcrunch.com)

* [Latest](/latest/)
* [Startups](/category/startups/)
* [Venture](/category/venture/)
* [Apple](/tag/apple/)
* [Security](/category/security/)
* [AI](/category/artificial-intelligence/)
* [Apps](/category/apps/)

* [Events](/events/)
* [Podcasts](/podcasts/)
* [Newsletters](/newsletters/)

Search

Submit

Site Search Toggle

Mega Menu Toggle

### Topics

[Latest](/latest/)

[AI](/category/artificial-intelligence/)

[Amazon](/tag/amazon/)

[Apps](/category/apps/)

[Biotech & Health](/category/biotech-health/)

[Climate](/category/climate/)

[Cloud Computing](/tag/cloud-computing/)

[Commerce](/category/commerce/)

[Crypto](/category/cryptocurrency/)

[Enterprise](/category/enterprise/)

[EVs](/tag/evs/)

[Fintech](/category/fintech/)

[Fundraisin